In [1]:
%pip install youtube-comments-scrapper

  Using cached youtube_comments_scrapper-1.0.1-py3-none-any.whl.metadata (6.0 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached selenium-4.26.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.27.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached sortedcontain

# Youtube Comment Scrapping

In [2]:
from youtube_comments_scraper import YouTubeCommentScraper

scraper = YouTubeCommentScraper(headless=True, timeout=10, scroll_pause_time=1.5, enable_logging=False, return_page_source=False)

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x648da99ff4e3 <unknown>
#1 0x648da972ec76 <unknown>
#2 0x648da9755757 <unknown>
#3 0x648da9754029 <unknown>
#4 0x648da9792ccc <unknown>
#5 0x648da979247f <unknown>
#6 0x648da9789de3 <unknown>
#7 0x648da975f2dd <unknown>
#8 0x648da976034e <unknown>
#9 0x648da99bf3e4 <unknown>
#10 0x648da99c33d7 <unknown>
#11 0x648da99cdb20 <unknown>
#12 0x648da99c4023 <unknown>
#13 0x648da99921aa <unknown>
#14 0x648da99e86b8 <unknown>
#15 0x648da99e8847 <unknown>
#16 0x648da99f8243 <unknown>
#17 0x74b55e694ac3 <unknown>


In [ ]:
Video_link = [
    https://www.youtube.com/watch?v=Rk1VmHk_kUo&list=PL2O3HdJI4voHNEv59SdXKRQVRZAFmwN9E&index=1
    ""
]

## Scrapping Youtube Video

## Preprocessing

## EDA

## Vectorization

## Conclusion